# 1. Import dependencies

In [2]:
import re
import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime, timedelta
from langchain_core.documents import Document
from typing import List
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

# 2. Scrape the latest articles (12 articles).

In [4]:
base_url = "https://web-data.api.hk01.com/v2/issues/10221/relatedBlock/0/"

headers = {
    'Referer': 'https://www.hk01.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36'
}

params = {
    'limit': 6, #每次六篇
    'offset': 0,  # 初始 offset
    'bucketId': '00000'
}

# save all the articles
all_articles = []

# 循环两次爬取12篇最新的文章
request_count = 0
max_requests = 2

print("开始获取文章列表...")
while True:
    print(f"正在请求第 {request_count + 1} 次 API...")
    
    # 发送 GET 请求
    response = requests.get(base_url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        # 提取 items
        items = data.get('items', [])
        next_offset = data.get('nextOffset', None)
        
        print(f"  - 获取到 {len(items)} 篇文章")
        
        # 将本次获取的文章添加到总列表
        all_articles.extend(items)
        
        # 检查是否有 nextOffset
        if next_offset is not None:
            # 更新 params 中的 offset 为 nextOffset
            params['offset'] = next_offset
            print(f"  - 更新 offset 为: {next_offset}")
        else:
            # 如果没有 nextOffset，说明数据已加载完毕
            print("  - 没有更多数据，停止请求。")
            break
        
        # 检查是否达到最大请求数
        request_count += 1
        if request_count >= max_requests:
            print(f"  - 达到最大请求数 {max_requests}，停止请求。")
            break
        
        # 添加短暂延迟，避免请求过于频繁
        time.sleep(0.5)
        
    else:
        print(f"  - 请求失败，状态码: {response.status_code}")
        print(response.text) # 打印错误信息
        break

print(f"\n--- 总共获取到 {len(all_articles)} 篇文章列表 ---\n")

开始获取文章列表...
正在请求第 1 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1770032606
正在请求第 2 次 API...
  - 获取到 6 篇文章
  - 更新 offset 为: 1769847711
  - 达到最大请求数 2，停止请求。

--- 总共获取到 12 篇文章列表 ---



# 3. This script will run by default every day at 0:00, therefore it will filter out news from yesterday.

In [5]:
# 计算昨天的日期（假设你的系统时间是香港时间 UTC+8）
yesterday = (datetime.now() - timedelta(days=1)).date()
# 转为 "YYYY-MM-DD" 字符串
yesterday_str = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

# 用于存储昨天的文章
yesterday_articles = []

output_filename = "daily_new_articles.md"
# 遍历所有获取到的文章
for i, item in enumerate(all_articles, 1):
    article_data = item.get('data', {})
    title = article_data.get('title', '无标题')
    url = article_data.get('canonicalUrl', '无链接')
    publish_timestamp = article_data.get('publishTime', None)

    # 转换时间戳为香港时间（如果有）
    publish_time_hk = "未知"
    pub_date_hk = None
    if publish_timestamp:
        try:
            dt_utc = datetime.utcfromtimestamp(publish_timestamp)
            dt_hk = dt_utc + timedelta(hours=8)  # 转为香港时间
            publish_time_hk = dt_hk.strftime("%Y-%m-%d %H:%M:%S")  # 格式化完整时间
            pub_date_hk = dt_hk.date()  # 提取日期部分
        except Exception as e:
            print(f"  - 时间戳转换失败: {publish_timestamp}, 错误: {e}")
            # 如果转换失败，无法判断日期，跳过此篇文章
            continue

    print(f"检查第 {i} 篇文章: {title[:30]}..., 发布日期: {pub_date_hk}")

    # 只处理昨天发布的文章
    if pub_date_hk == yesterday:
        print(f"  -> 符合条件，加入处理列表。")
        yesterday_articles.append({
            'index': len(yesterday_articles) + 1, # 重新编号
            'item': item,
            'title': title,
            'url': url,
            'publish_time_hk': publish_time_hk
        })
    else:
        print(f"  -> 不是昨天的新闻，跳过。")

检查第 1 篇文章: 尖沙咀見這東西不要撿！港男遇「觸碰型陷阱」騙財　機智1招脫困..., 发布日期: 2026-02-05
  -> 不是昨天的新闻，跳过。
检查第 2 篇文章: 緬北四大家族覆滅　明家白家伏法　中國用三年徹底剷除電騙毒瘤？..., 发布日期: 2026-02-04
  -> 符合条件，加入处理列表。
检查第 3 篇文章: 10萬元器械植入體內離奇消失？河南醫生詐騙94名患者判囚12..., 发布日期: 2026-02-04
  -> 符合条件，加入处理列表。
检查第 4 篇文章: 新型碰瓷黨｜醫生及索償人共4人涉騙被捕　警檢名錶金器拖走的士..., 发布日期: 2026-02-04
  -> 符合条件，加入处理列表。
检查第 5 篇文章: 日本首例！歌舞伎町王牌牛郎交友App非法拉客被捕　扮IT男騙..., 发布日期: 2026-02-04
  -> 符合条件，加入处理列表。
检查第 6 篇文章: 網騙2026：當攻擊進入Agentic AI時代，CIO 應..., 发布日期: 2026-02-04
  -> 符合条件，加入处理列表。
检查第 7 篇文章: 一周110宗經Carousell網購騙案呃過千萬　有人賣$1..., 发布日期: 2026-02-02
  -> 不是昨天的新闻，跳过。
检查第 8 篇文章: 網上情緣騙案｜扮女人氹落疊稱贈遺產　冒警二次詐騙　共呃440..., 发布日期: 2026-02-02
  -> 不是昨天的新闻，跳过。
检查第 9 篇文章: 警反詐騙拘682人涉$6.2億　詐騙集團聘洗黑錢集團洗白$4..., 发布日期: 2026-02-02
  -> 不是昨天的新闻，跳过。
检查第 10 篇文章: 新型詐騙？2男女稱大學玩遊戲求傳自拍照　網民點出破綻：係騙案..., 发布日期: 2026-02-02
  -> 不是昨天的新闻，跳过。
检查第 11 篇文章: 緬北電騙白家案　白所成一審後病亡　白應蒼等4人伏法..., 发布日期: 2026-02-02
  -> 不是昨天的新闻，跳过。
检查第 12 篇文章: 內地23歲男廚師冒充殲-16機師　與上百名女子戀愛圖呃錢被捕..., 发布日期: 2026-02-01
  -> 不是昨天的新闻，跳过。


# 4. Save all yesterday's articles

In [6]:
# 如果没有找到昨天的文章，则不创建文件
if not yesterday_articles:
    print(f"\n--- 没有找到 {yesterday} 的文章，跳过写入文件。---")
else:
    print(f"\n--- 找到 {len(yesterday_articles)} 篇昨天的文章，开始写入文件... ---")
    # 现在开始写入 Markdown 文件
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(f"共 {len(yesterday_articles)} 篇（仅 {yesterday} 发布）\n\n")

        for article_obj in yesterday_articles:
            i = article_obj['index']
            item = article_obj['item']
            title = article_obj['title']
            url = article_obj['url']
            publish_time_hk = article_obj['publish_time_hk']

            print(f"正在处理昨天的第 {i} 篇文章: {title[:30]}...")

            try:
                # 发送 GET 请求获取文章页面内容
                response = requests.get(url)

                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # 提取正文 (找到所有 class 包含 "whitespace-pre-wrap break-words" 的 <p> 标签)
                    paragraphs = soup.find_all('p', class_='whitespace-pre-wrap break-words')
                    content = "\n\n".join([p.get_text(strip=True) for p in paragraphs])

                    # 写入 Markdown 格式（添加发布时间）
                    f.write(f"## [{i}. {title}]({url})\n\n")
                    f.write(f"**发布时间（香港时间）**: {publish_time_hk}\n\n")  # 添加时间信息
                    f.write(f"{content}\n\n---\n\n") # 使用 --- 作为文章分隔符

                else:
                    print(f"  - 请求文章失败，状态码: {response.status_code}")
                    # 即使请求失败，也写入标题、链接和时间，标记错误
                    f.write(f"## [{i}. {title}]({url})\n\n")
                    f.write(f"**发布时间（香港时间）**: {publish_time_hk}\n\n")  # 添加时间信息
                    f.write(f"[Error: Failed to fetch content, status code {response.status_code}]\n\n---\n\n")

            except Exception as e:
                print(f"  - 抓取文章时出错: {e}")
                # 即使出错，也写入标题、链接和时间，标记错误
                f.write(f"## [{i}. {title}]({url})\n\n")
                f.write(f"**发布时间（香港时间）**: {publish_time_hk}\n\n")  # 添加时间信息
                f.write(f"[Error: {str(e)}]\n\n---\n\n")

    print(f"\n--- Markdown 文件 '{output_filename}' 写入完成！仅包含 {yesterday} 的 {len(yesterday_articles)} 篇文章。---")


--- 找到 5 篇昨天的文章，开始写入文件... ---
正在处理昨天的第 1 篇文章: 緬北四大家族覆滅　明家白家伏法　中國用三年徹底剷除電騙毒瘤？...
正在处理昨天的第 2 篇文章: 10萬元器械植入體內離奇消失？河南醫生詐騙94名患者判囚12...
正在处理昨天的第 3 篇文章: 新型碰瓷黨｜醫生及索償人共4人涉騙被捕　警檢名錶金器拖走的士...
正在处理昨天的第 4 篇文章: 日本首例！歌舞伎町王牌牛郎交友App非法拉客被捕　扮IT男騙...
正在处理昨天的第 5 篇文章: 網騙2026：當攻擊進入Agentic AI時代，CIO 應...

--- Markdown 文件 'daily_new_articles.md' 写入完成！仅包含 2026-02-04 的 5 篇文章。---


# 5. Convert the article into a document and store it in the existing vector database.

In [7]:
file_path = "daily_new_articles.md"
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 按 "---" 分割文章（支持前后有空行）
articles = [a.strip() for a in re.split(r'\n-{3,}\n', content) if a.strip()]

print(f"共加载 {len(articles)} 篇诈骗新闻")

共加载 5 篇诈骗新闻


In [ ]:
from langchain_core.documents import Document
docs = []
for i, art in enumerate(articles):
    # 提取标题（可选优化）
    lines = art.splitlines()
    title = lines[0].strip().lstrip('#').strip() if lines else f"文章 #{i+1}"
    doc = Document(
        page_content=art,
        metadata={
            "source": "daily_new_articles.md",
            "article_id": i + 1,
            "title": title,
            "date": yesterday_str
        }
    )
    docs.append(doc)

In [ ]:
class LocalEmbeddings:
    def __init__(self, model_name="shibing624/text2vec-base-chinese"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, normalize_embeddings=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([text], normalize_embeddings=True)[0].tolist()

In [ ]:
embedding_model = LocalEmbeddings()
vectorstore = Chroma(
    persist_directory="./chroma_hk01_scam_db",
    embedding_function=embedding_model
)
vectorstore.add_documents(docs)

print("更新完成！RAG 系统现在包含最新诈骗新闻。")
print(f"写入后文档总数: {vectorstore._collection.count()}")